## Systematic modelling of surface deformation at active volcanoes
### ethz-02-03-01

This application takes Surface displacement retrieved with DInSAR at active volcanoes to retrieve a first orde estimate of the volume change in the subsurface.

### <a name="service">Service definition

In [ ]:
service = dict([('title', 'Systematic modelling of surface deformation at active volcanoes'),
                ('abstract', 'Systematic modelling of surface deformation at active volcanoes'),
                ('id', 'ewf-ethz-02-03-01')])

In [ ]:
coordinates = dict([('id', 'coordinates'),
                    ('title', 'coordinates'),
                    ('abstract', 'Approx Coordinates fo the co-seismic signal (Lat, Lon)'),
                    ('value', '-7.426,110.441')])

In [ ]:
buffer_aoi = dict([('id', 'buffer_aoi'),
                   ('title', 'buffer_aoi'),
                   ('abstract', 'Buffer AOI (degrees)'),
                   ('value', '0.075')])

In [ ]:
downsampling = dict([('id', 'downsampling'),
                     ('title', 'downsampling'),
                     ('abstract', 'Downsampling for speed (0.05-1)'),
                     ('value', '0.2')])

In [ ]:
los_angle = dict([('id', 'los_angle'),
                  ('title', 'los_angle'),
                  ('abstract', 'LOS angles of the satellite (incidence33-43 for S1, azimuth, +15 Descending, -15 Ascending)'),
                  ('value', '40,-15')])

In [ ]:
_T2Username = dict([('id', '_T2Username'),
                    ('title', 'T2Username'),
                    ('abstract', 'Terradue username'),
                    ('value', 'apisani')])

In [ ]:
_T2ApiKey = dict([('id', '_T2ApiKey'),
                  ('title', 'T2ApiKey'),
                  ('abstract', 'Terradue api_key'),
                  ('value', 'AKCp5cbcp6jQihbsinxxoD9s78vDvdspUAFhb1PAJB5y3r7xDEAGU7rZ2gG9YzjQhFFWuMk9V')])

### Runtime parameter definition

**Input identifier**

Product identifier

In [ ]:
input_identifier = ('F4636C8462A88D225393E23281C011ED477B22A7')

**Input reference**

Catalogue reference

In [ ]:
input_reference = ('https://catalog.terradue.com/better-ethz-02-01-01/search?format=atom&uid=F4636C8462A88D225393E23281C011ED477B22A7')

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = '/workspace/data'

#### Import the packages required for processing the data

In [ ]:
import os
import sys
import subprocess

sys.path.append('/application/notebook/libexec/') 
sys.path.append(os.getcwd())

from ellip_helpers import create_metadata

import gdal
import cioppy
ciop = cioppy.Cioppy()

### Product check

In [ ]:
search = ciop.search(end_point=input_reference,
                     params=[('do', 'terradue')],
                     output_fields='enclosure, startdate, enddate, wkt',
                     model='EOP')[0]

enclosure = search['enclosure']
product = os.path.basename(enclosure)

In [ ]:
product_path = os.path.join(data_path, product)

print 'Searching:', product_path

if os.path.isfile(product_path):
    print "Product {} Retrieved".format(product)
else:
    raise(Exception("Product {} with reference {} not found in data path {}".format(product, input_reference, data_path)))

In [ ]:
ciop.copy(product_path, '.')

### <a name="workflow">Workflow

In [ ]:
if 'LD_LIBRARY_PATH' not in os.environ.keys():
    os.environ['LD_LIBRARY_PATH'] = '/opt/v94/runtime/glnxa64:/opt/v94/bin/glnxa64:/opt/v94/sys/os/glnxa64:/opt/v94/extern/bin/glnxa64'
else:
    os.environ['LD_LIBRARY_PATH'] = '/opt/v94/runtime/glnxa64:/opt/v94/bin/glnxa64:/opt/v94/sys/os/glnxa64:/opt/v94/extern/bin/glnxa64:' + os.environ['LD_LIBRARY_PATH']
    
import run_inverse_model

#### Creating the input_modeling  file

In [ ]:
with open('input_modeling.txt', 'wb') as file:
    file.write('{}\n'.format(product))
    file.write('{} {}\n'.format(coordinates['value'].split(',')[0], coordinates['value'].split(',')[1]))
    file.write('{}\n'.format(buffer_aoi['value']))
    file.write('{}\n'.format(downsampling['value']))
    file.write('{} {}\n'.format(los_angle['value'].split(',')[0], los_angle['value'].split(',')[1]))

In [ ]:
with open('input_modeling.txt') as file:
    print file.read()

In [ ]:
command = 'import run_inverse_model; mr = run_inverse_model.initialize(); mr.run_inverse_model(\"input_modeling.txt\", nargout=0)'

options = ['python',
           '-c',
           command,
          ]

print options

In [ ]:
p = subprocess.Popen(options,
                     stdout=subprocess.PIPE,
                     stdin=subprocess.PIPE,
                     stderr=subprocess.PIPE)

res, err = p.communicate()

print(res)
print(err)

#### Read the geo_transform and the projection from the input

In [ ]:
src = gdal.Open(product_path)

geo_transform = src.GetGeoTransform()
projection = src.GetProjection()

src.FlushCache()

In [ ]:
os.listdir('./')

#### Get the output file list

In [ ]:
os.remove(product)
os.remove("input_modeling.txt")

In [ ]:
os.listdir('./')

In [ ]:
output_file = list()

for file in os.listdir('.'):
    if '.mat' in file:
        output_file.append(file)
        
output_file

In [ ]:
os.listdir('./')

#### Create the geo_transform and the projection file txt

In [ ]:
trf = {'geo_transform' : geo_transform,
        'projection' : projection}

with open('geo_transform_projection.txt', 'wb') as file:
    file.write(str(trf))
    file.close()

In [ ]:
with open('geo_transform_projection.txt') as f:
    print f.read()

In [ ]:
os.listdir('./')

for file in output_file:
    ds = gdal.Open(file, gdal.OF_UPDATE)
    
    ds.SetGeoTransform(geo_transform)
    ds.SetProjection(projection)
    
    ds.FlushCache()
    
    print gdal.Info(file)

In [ ]:
metadata = dict()

metadata['startdate'] = search['startdate']
metadata['enddate'] = search['enddate']
metadata['wkt'] = search['wkt']

metadata

In [ ]:
os.listdir('./')

In [ ]:
output_file.append('geo_transform_projection.txt')

In [ ]:
for file in output_file:
    print os.path.splitext(file)[0]

    metadata['identifier'] = os.path.splitext(file)[0]
    metadata['title'] = metadata['identifier']
    create_metadata(metadata, metadata['identifier'])

In [ ]:
os.listdir('./')

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.